In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import convolve
from scipy.ndimage import binary_dilation
import matplotlib.animation as animation
from matplotlib import colors

In [ ]:
# parameters
n = 100 
m = 100 
house_range = 5
num_steps = 10
income_values = [0.1, 0.5, 1.0]
income_probabilities = [0.5, 0.3, 0.2]

amenities_names_fixed = ['shop', 'supermarket', 'post office', 'gym', 'school']

In [ ]:
def create_edinburgh_map(n, m):
    city_map = np.full((n, m), None, dtype=object)

    I, J = np.indices((n, m))
    parks = np.zeros((n, m), dtype=bool)

    # oval that can be roatated to model the empty cells
    
    def rotated_oval(centre_i, centre_j, ri, rj, theta_deg):
        theta = np.deg2rad(theta_deg)
        Y = I - centre_i   # rows
        X = J - centre_j   # cols
        Xr = X * np.cos(theta) + Y * np.sin(theta)
        Yr = -X * np.sin(theta) + Y * np.cos(theta)
        return (Xr / rj) ** 2 + (Yr / ri) ** 2 <= 1.0

    meadows = rotated_oval(
        centre_i=82,   # move it down a little
        centre_j=53,
        ri=4,          # slightly thinner
        rj=26,         # a bit longer
        theta_deg=-10
    )

    # Bruntsfield Links: more clearly south-west of the Meadows
    links_main = rotated_oval(
        centre_i=88,   # a bit further south
        centre_j=30,   # slightly further west
        ri=5,
        rj=18,
        theta_deg=-20
    )

    # Tail joining Links up towards the western end of the Meadows
    tail = (
        (I >= 84) & (I <= 94) &
        (J >= 22) & (J <= 36) &
        (J <= -1.0 * (I - 94) + 36)
    )

    parks |= meadows | links_main | tail

    #other parks/emtpy cells
    psg_west = rotated_oval(centre_i=32, centre_j=42, ri=5,   rj=16,  theta_deg=-8)
    psg_east = rotated_oval(centre_i=34, centre_j=60, ri=4.5, rj=18,  theta_deg=2)
    castle_slope = rotated_oval(centre_i=28, centre_j=48, ri=6, rj=14, theta_deg=-10)
    calton = rotated_oval(centre_i=32, centre_j=84, ri=7, rj=12, theta_deg=0)

    holy_centre_i, holy_centre_j, holy_radius = 60, 100, 15
    holy_circle = ((I - holy_centre_i) ** 2 + (J - holy_centre_j) ** 2) <= holy_radius ** 2
    holyrood = holy_circle

    uni = rotated_oval(centre_i=54, centre_j=54, ri=4, rj=9, theta_deg=0)

    parks |= psg_west | psg_east | castle_slope | calton | holyrood | uni

    #road/emtpy cells
    queen_st   = (I >= 18) & (I <= 28) & (np.abs(J - (0.25 * I + 28)) <= 1)
    george_st  = (I >= 20) & (I <= 30) & (np.abs(J - (0.25 * I + 34)) <= 1)
    princes_st = (I >= 22) & (I <= 32) & (np.abs(J - (0.25 * I + 40)) <= 1)
    lothian    = (I >= 24) & (I <= 80) & (np.abs(J - (0.03 * I + 38)) <= 1)
    leith      = (I >= 18) & (I <= 60) & (np.abs(J - (0.45 * I + 30)) <= 1)
    lauriston  = (I >= 48) & (I <= 49) & (J >= 24) & (J <= 80)
    melville   = (I >= 78) & (I <= 79) & (J >= 18) & (J <= 90)
    brunts_rd  = (I >= 60) & (I <= 95) & (np.abs(J - (0.08 * I + 24)) <= 1)
    fountain   = (I >= 26) & (I <= 90) & (np.abs(J - (0.02 * I + 18)) <= 1)

    royal_mile = (np.abs(J - (0.6 * I - 5)) <= 1) & (I >= 32) & (I <= 60)
    royal_bend = (I >= 44) & (I <= 52) & (np.abs(J - (0.45 * I + 10)) <= 1)
    cowgate    = (np.abs(J - (0.6 * I - 10)) <= 1) & (I >= 40) & (I <= 65)
    main_between = (
        (I >= 40) & (I <= 99) &   # only in the gap band
        (J >= 79) & (J <= 80)     # 3-cell-wide vertical strip
    )

    roads = (queen_st | george_st | princes_st |
             lothian | leith | lauriston |
             melville | brunts_rd | fountain |
             royal_mile | royal_bend | cowgate| main_between)

    parks |= roads

# making smaller sreets for it to be more realistic
    for col in range(10, 92, 8):
        col_mask = (J == col) & (~parks) & (I >= 6) & (I <= 95)
        parks |= col_mask

    for row in range(6, 90, 8):
        row_mask = (I == row) & (~parks) & (J >= 8) & (J <= 95)
        parks |= row_mask


    stockbridge = rotated_oval(centre_i=14, centre_j=40, ri=4, rj=10, theta_deg=-5)
    inverleith  = rotated_oval(centre_i=10, centre_j=60, ri=4, rj=12, theta_deg=5)
    parks |= stockbridge | inverleith


    city_map[parks] = 'park'


    north_shops  = (I >= 8) & (I <= 14) & (J >= 30) & (J <= 70) & (~parks)
    north_super  = (I >= 12) & (I <= 15) & (J >= 48) & (J <= 52) & (~parks)
    north_school = (I >= 10) & (I <= 14) & (J >= 34) & (J <= 40) & (~parks)
    city_map[north_shops]  = 'shop'
    city_map[north_super]  = 'supermarket'
    city_map[north_school] = 'school'


    city_map[34:42, 60:70] = 'station'
    city_map[26:30, 60:66] = 'park'
    city_map[36:42, 44:52] = 'park'


    theta = np.deg2rad(-8.0)
    cx, cy = 25.0, 55.0
    X = J - cy
    Y = I - cx
    Xr = X * np.cos(theta) - Y * np.sin(theta)
    Yr = X * np.sin(theta) + Y * np.cos(theta)

    new_town_shops = (I >= 20) & (I <= 36) & (np.abs(Yr) <= 6) & (Xr >= -26) & (Xr <= 26)
    city_map[new_town_shops & (~parks)] = 'shop'

    school_blocks = [
        ((78, 44), (83, 52)),
        ((60, 58), (64, 64)),
        ((60, 26), (64, 32)),
        ((68, 40), (73, 48)),
        ((38, 34), (43, 42)),
    ]
    for (i1, j1), (i2, j2) in school_blocks:
        block_mask = (I >= i1) & (I <= i2) & (J >= j1) & (J <= j2)
        city_map[block_mask & (~parks)] = 'school'


    qm_gym = (I >= 50) & (I <= 54) & (J >= 48) & (J <= 54)
    city_map[qm_gym & (~parks)] = 'gym'

    marchmont_shops    = (I >= 84) & (I <= 90) & (J >= 44) & (J <= 62)
    bruntsfield_shops  = (I >= 82) & (I <= 90) & (J >= 14) & (J <= 30)
    city_map[marchmont_shops & (~parks)]   = 'shop'
    city_map[bruntsfield_shops & (~parks)] = 'shop'


    brunts_super    = (I >= 52) & (I <= 55) & (J >= 30) & (J <= 34)
    newington_super = (I >= 84) & (I <= 88) & (J >= 54) & (J <= 58)
    new_town_super  = (I >= 23) & (I <= 27) & (J >= 54) & (J <= 58)
    city_map[brunts_super & (~parks)]    = 'supermarket'
    city_map[newington_super & (~parks)] = 'supermarket'
    city_map[new_town_super & (~parks)]  = 'supermarket'

    # scattered amenities
    amenity_positions = [
        (48, 46, 'supermarket'), 
        (52, 32, 'supermarket'),  
        (84, 55, 'supermarket'),  
        (62, 80, 'supermarket'), 
        (24, 56, 'supermarket'),  
        (26, 64, 'supermarket'),  

        (46, 42, 'shop'),
        (60, 38, 'shop'), 
        (60, 28, 'shop'), 
        (52, 52, 'shop'),  
        (82, 44, 'shop'), 
        (30, 50, 'shop'), 
        (34, 60, 'shop'),   

        (52, 40, 'post office'),  
        (80, 56, 'post office'),
        (60, 36, 'post office'),  

        (48, 50, 'gym'), 
        (58, 62, 'gym'), 
        (54, 44, 'gym'),  

        # extra New Town amenities
        (24, 55, 'supermarket'),  
        (26, 62, 'supermarket'),   
        (25, 50, 'post office'),   
        (23, 47, 'gym'),
    ]
    for i, j, name in amenity_positions:
        if 0 <= i < n and 0 <= j < m:
            city_map[i, j] = name

    return city_map


# build map
the_map = create_edinburgh_map(n, m)
print("Amenity / city map created with shape:", the_map.shape)


In [ ]:
#adding in the SIMD data from other notebook
A_amen_simd   = np.load("./A_amenities.npy")
A_score_simd  = np.load("./A_matrix.npy")        
income_rank   = np.load("income_rank_matrix.npy")

print("A_amen_simd:", A_amen_simd.shape)
print("A_score_simd:", A_score_simd.shape)
print("income_rank:", income_rank.shape)


In [ ]:
# making the SIMD data income rank to the poor/middle/rich used here
#0.1 poor
#0.5 middle
# 1 rich

# Normalise ranks
r_min = income_rank.min()
r_max = income_rank.max()
r_norm = (income_rank - r_min) / (r_max - r_min + 1e-9)

# Split into terciles
low_thresh  = 1/3
high_thresh = 2/3

A_incomes_real = np.zeros_like(r_norm)

A_incomes_real[r_norm <= low_thresh]  = 0.1  # more deprived
A_incomes_real[(r_norm > low_thresh) & (r_norm <= high_thresh)] = 0.5
A_incomes_real[r_norm > high_thresh]  = 1.0  # least deprived

print("SIMD-based income grid:", A_incomes_real.shape,
      "unique values:", np.unique(A_incomes_real))


In [ ]:


def createlat_from_SIMD(n, m, A_incomes_real):
    """
    Create households and initial values using the SIMD-derived income grid.
    the_map is your Edinburgh city map created earlier.
    This does not change any existing functions.
    """
    households = np.empty((n, m), dtype=object)
    value      = np.zeros((n, m), dtype=float)

    for i in range(n):
        for j in range(m):
            cell_type = the_map[i, j]   # uses your existing Edinburgh map

            # Parks and roads: keep them empty with zero house value
            if cell_type in ("park", "road"):
                households[i, j] = None
                value[i, j] = 0.0
                continue

            # All other cells: place a household whose income comes from SIMD
            income_ij = float(A_incomes_real[i, j])
            households[i, j] = Household(income_ij)

            # Simple initial price = income (you can comment on this choice in the write-up)
            value[i, j] = income_ij

    return households, value


In [ ]:
class Household():
    def __init__(self, income):
        ## Initialise
        self.income = income,
        self.amenities = {}

        ## Fill self.amenities
        # choose random religion/language/age category
        self.amenities['religion'] = [self.__choose_religion__(), None]
        self.amenities['language'] = [self.__choose_language__(), None]
        self.amenities['age_category'] = [self.__choose_age_category__(), None]

        # importance weights - can depend on the religion/language/age category chosen
        self.amenities['religion'][1] = self.__get_importance__('religion')
        self.amenities['language'][1] = self.__get_importance__('language')
        self.amenities['age_category'][1] = self.__get_importance__('age_category')

        # school importance depends on age_category
        self.amenities['school'] = self.__get_school_importance__()

        # fixed V_amenities weights (shop/supermarket/post office/gym)
        self.amenities['shop'] = self.__get_importance__('shop')
        self.amenities['supermarket'] = self.__get_importance__('supermarket')
        self.amenities['post office'] = self.__get_importance__('post office')
        self.amenities['gym'] = self.__get_importance__('gym')

    ### Initialisation methods
    def __get_importance__(self, amenity_name=None):
        if amenity_name=='shop':
            return 0.7
        elif amenity_name=='supermarket':
            return 0.9
        elif amenity_name=='post office':
            return 0.4
        elif amenity_name=='gym':
            if self.amenities['age_category'][0] in ['Single(s)', 'Young couple', 'Other']:
                return np.random.rand() * 0.5 + 0.5  # medium to high importance between 0.5 and 1.0
            elif self.amenities['age_category'][0] in ['Middle-aged couple', 'Family']:
                return np.random.rand() * 0.4 + 0.3  # low to medium importance between 0.3 and 0.7
            else:  # Elderly couple
                return np.random.rand() * 0.3  # low importance between 0.0 and 0.3

        return np.random.rand()  # catches other cases when amenity_name is not specified
    
    def __choose_religion__(self):
        religions = ['Christian', 'No religion', 'Muslim', 'Hindu', 'Sikh', 'Buddhist', 'Jewish', 'Other', 'Not stated']
        
        '''numbers = np.array([1717871+841053+291275, 1941116, 76737, 16379, 12795, 9055, 5997, 15196, 0])  # https://www.scotlandscensus.gov.uk/census-results/at-a-glance/religion/ - Accessed 17th November
        numbers[-1] = sum(numbers)/(100-7)*7
        probabilities = numbers / numbers.sum()'''
        probabilities = [0.53793995, 0.36636174, 0.01448316, 0.00309133, 0.0024149, 0.00170902, 0.00113186, 0.00286806, 0.06999998]

        return np.random.choice(religions, p=probabilities)
    
    def __choose_language__(self):
        languages = ['English', 'Other'] 
        
        # https://www.scotlandscensus.gov.uk/census-results/at-a-glance/languages/  - Accessed 17th November
        probabilities = [0.926, 1-0.926]  # 92.6% spoke only English in their homes

        return np.random.choice(languages, p=probabilities)
    
    def __choose_age_category__(self):
        ages = ['Single(s)', 'Young couple', 'Middle-aged couple', 'Elderly couple', 'Family', 'Other']  # Family has children, all couples do not currently have children
        
        probabilities = [0.2, 0.15, 0.1, 0.15, 0.3, 0.1]  # Made up for now

        return np.random.choice(ages, p=probabilities)
    
    def __get_school_importance__(self):

        if self.amenities['age_category'][0] in ['Family']:  # has children
            return np.random.rand() * 0.1 + 0.9  # high importance between 0.9 and 1.0
        elif self.amenities['age_category'][0] in ['Young couple', 'Elderly couple']:  # may plan to have children soon or have grandchildren they babysit
            return np.random.rand() * 0.3 + 0.5  # medium importance between 0.5 and 0.8
        else:
            return 0  # no importance
        
    ### Setters and getters - GitHub Copilot was used to generate these methods
    def set_income(self, new_income):
        self.income = (new_income,)
    
    def get_income(self):
        return self.income[0]  # for some reason income is stored as a tuple
    
    def get_amenities(self):
        """Return a shallow copy of the amenities dictionary."""
        return self.amenities.copy()

    def set_amenities(self, new_amenities):
        """Replace the entire amenities dictionary."""
        self.amenities = new_amenities

    def get_amenity(self, name):
        """Return the raw stored value for an amenity (could be list or numeric)."""
        return self.amenities.get(name, None)

    def set_amenity(self, name, value):
        """Set the raw stored value for an amenity (overwrites existing entry)."""
        self.amenities[name] = value

    def get_amenity_category(self, name):
        """
        For amenities stored as [category, importance] (e.g. 'religion', 'language', 'age_category'),
        return the category. For scalar amenities (e.g. 'school' or V), return None.
        """
        val = self.amenities.get(name)
        if isinstance(val, list) and len(val) >= 1:
            return val[0]
        return None

    def get_amenity_importance(self, name):
        """
        Return the importance/weight associated with an amenity.
        For entries stored as [category, importance] returns the importance.
        For scalar amenities returns the scalar value.
        """
        val = self.amenities.get(name)
        if isinstance(val, list) and len(val) >= 2:
            return val[1]
        return val  # might be float or None

    def set_amenity_importance(self, name, importance):
        """
        Set the importance/weight for an amenity.
        If the amenity is stored as [category, importance], updates the importance.
        Otherwise replaces the amenity value with the provided importance.
        """
        val = self.amenities.get(name)
        if isinstance(val, list) and len(val) >= 2:
            self.amenities[name][1] = importance
        else:
            self.amenities[name] = importance

    def to_dict(self):
        """Return a snapshot dictionary representation of the household (income + amenities)."""
        return {"income": self.income, "amenities": self.get_amenities()}
    
    ### Additional methods

    def get_amenity_importances_sum(self):
        return self.get_amenity_importance('religion') + self.get_amenity_importance('language') + self.get_amenity_importance('age_category') + self.get_amenity_importance('school')
    

class PropertyValue():
    def __init__(self, price):
        self.price = price
        self.amenities = {  # amenity in neighbourhood True/False
            'shop': False,
            'supermarket': False,
            'post office': False,
            'gym': False,
            'school': False
        }

    ### Setters and getters
    def set_price(self, new_price):
        self.price = new_price

    def get_price(self):
        return self.price

    def get_amenity_bool(self, name):
        if self.amenities[name] == True:
            return 1
        else:
            return 0

    def set_amenity(self, name, boolean):
        self.amenities[name] = boolean


def _gen_amenity_matrix(prices, my_map=the_map):
    '''
    Create V matrix - combining house prices and their additional value due to amenities
    '''
    # These amenities are positive for ALL households

    amenities = np.zeros((n,m), dtype=object)

    # Fill with prices and zero amenities first
    for i in range(0, n):
        for j in range(0, m):
            amenities[i, j] = PropertyValue(float(prices[i, j]))
            
    for i in range(0, n):
        for j in range(0, m):

            # if within neighbourhood of each amenity, populate amenities matrix with += the amenity weight
            for am_name in amenities_names_fixed:

                if my_map[i][j] == am_name:
                    # Modify A to have constant amenities at 'locations' - display all same colour so set value to 0.0001
                    amenities[i][j].set_price(0.0)  # amenity locations have zero price

                    for distance_x in range(-house_range, house_range+1):
                        for distance_y in range(-house_range, house_range+1):
                            if distance_x == 0 and distance_y == 0:
                                continue  # people cannot live at the site of an amenity so zero everything
                            else:
                                new_x = i+distance_x
                                new_y = j+distance_y
                                if 0 <= new_x < n and 0 <= new_y < m:  # neighbour is in city - m and n may be wrong way around here
                                    #print(amenities[new_x][new_y])
                                    amenities[new_x][new_y].set_amenity(am_name, True)  # this amenity is in the neighbourhood of each house


    #print("Amenities Matrix:\n", amenities)
    print("Amenities Matrix:\n", get_prices_matrix(amenities))
    return amenities

def gen_amenity_matrix(prices, my_map, amenities_names_fixed=amenities_names_fixed, 
                       house_range=house_range):
    n, m = prices.shape
    
    # 1. Vectorized Object Creation
    # Instead of a loop, use np.vectorize (or list comprehension) to fill the grid initially.
    # We define a helper to instantiate the class.
    v_create = np.vectorize(lambda p: PropertyValue(float(p)), otypes=[object])
    amenities = v_create(prices)

    # 2. Handle "Price is zero at amenity location"
    # Create a mask for ALL amenities at once
    is_any_amenity = np.isin(my_map, amenities_names_fixed)
    
    # Update prices for all amenity locations in one pass
    # (We iterate only the specific objects that need updating, not the whole grid)
    for prop in amenities[is_any_amenity]:
        prop.set_price(0.0)

    # 3. Handle Neighborhoods
    # Create the square kernel using np.outer (as requested)
    # A 1D array of True of size (2*range + 1)
    k_1d = np.ones(2 * house_range + 1, dtype=bool)
    # Outer product creates the 2D square window
    kernel = np.outer(k_1d, k_1d) 

    for am_name in amenities_names_fixed:
        # Find exactly where this specific amenity is
        exact_locations = (my_map == am_name)
        
        # Skip if this amenity doesn't exist in the map
        if not np.any(exact_locations):
            continue

        # "Dilate" the locations: spreading True values to neighbors
        # This replaces the nested x/y loops
        neighborhood_mask = binary_dilation(exact_locations, structure=kernel)
        
        # Logic check: Your original code excluded the amenity center (distance_x=0, distance_y=0)
        # from having the 'set_amenity' flag.
        # So we subtract the exact locations from the neighborhood.
        affected_houses_mask = neighborhood_mask & (~exact_locations)
        
        # Update the objects
        # accessing amenities[mask] gives us a flat array of just the objects we need to modify
        for prop in amenities[affected_houses_mask]:
            prop.set_amenity(am_name, True)

    return amenities

def get_incomes_matrix(A):
    n = A.shape[0]
    m = A.shape[1]
    incomes_matrix = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            incomes_matrix[i,j] = A[i,j].get_income()
    return incomes_matrix

def get_prices_matrix(V):
    n = V.shape[0]
    m = V.shape[1]
    prices_matrix = np.zeros((n,m))
    for i in range(n):
        for j in range(m):
            prices_matrix[i,j] = V[i,j].get_price()
    return prices_matrix

def populate_new_prices(old_V, new_prices):
    V = old_V.copy()
    for i in range(n):
        for j in range(m):
            V[i,j].set_price(new_prices[i,j])
    return V

def house_update(neighbourhood_size, A, V):
    # Update house values

    print("neighbourhood_size", neighbourhood_size)

    old_V = V.copy()
    z = 0.01 # coefficient value (change as necessary) # z is lambda...    

    # Define Neighbourhood area size
    neighbourhood = np.ones((neighbourhood_size,neighbourhood_size))

    amenity_locations = np.argwhere(get_prices_matrix(V) == 0.0001)
    # Computes sum of neighbourhood house values and number of neighbours using convolution
    # constant mode with cval = 0 treats anything outside of edges of matrix as zero
    # Search up convolve function to understand this part
    neighbourhood_sum = convolve(get_prices_matrix(old_V), neighbourhood, mode='constant', cval = 0)
    neighbourhood_count = convolve(np.ones((n,m)), neighbourhood, mode='constant', cval = 0)


    # Use equation given in worksheet to calculate new values of houses
    new_prices = np.round(get_incomes_matrix(A) + z * (neighbourhood_sum / neighbourhood_count) + get_prices_matrix(V))
    # keep amenity locations fixed at 0.0
    for (ai, aj) in amenity_locations:
        new_prices[ai, aj] = 0.0
    V = populate_new_prices(V, new_prices)

    #print("Sum of Neighbourhood House Values:\n", neighbourhood_sum) 
    #print("Number of Neighbours:\n", neighbourhood_count)
    #print("Updated House Value:\n", V)
    return A, V

def delta(x1, x2, y1, y2):
    '''
    x measures how much you want it
    y measures how much it costs
    '''
    d = (x1 - y1)**2 + (x2 - y2)**2 - (x1 - y2)**2 - (x2 - y1)**2 
    return d

def get_proportion_of_neighbours(A, household, loc, category, amenity_name):
    '''
    A is the matrix of households
    household is the Household object at that location
    loc is the location of the house (i,j)
    category is the category we are measuring proportion of
    '''
    global house_range  # neighbourhood size
    i = loc[0]
    j = loc[1]

    count_same = 0
    count_total = 0

    # Check neighbours in a 3x3 grid around the house (excluding itself)
    boundary_idx = house_range // 2
    diffs = sorted([s for s in range(-boundary_idx, boundary_idx)])
    for di in diffs:
        for dj in diffs:
            ni = i + di
            nj = j + dj
            if (di == 0 and dj == 0):
                continue  # skip itself
            else:
                if 0 <= ni < n and 0 <= nj < m:
                    neighbour_household = A[ni][nj]
                    neighbour_category = neighbour_household.get_amenity_category(amenity_name)
                    if neighbour_category == category:
                        count_same += 1
                    count_total += 1

    if count_total == 0:
        return 0  # avoid division by zero; no neighbours

    proportion = count_same / count_total
    return proportion

def get_amenities_value(A, loc1, val1, household1, loc2, val2, household2):
    '''
    loc is the location of the house (i,j)
    val is the value of the fixed amenities at that location
    household is the Household object at that location
    '''

    amenities_value1 = 0
    amenities_value2 = 0

    amenity_names_fixed = ['shop', 'supermarket', 'post office', 'gym', 'school']
    for amenity_name in amenity_names_fixed:

        # Notice these will be negative if the swap is bad for that household
        how_good_a_swap_for_h1 = household1.get_amenity_importance(amenity_name)*(val2.get_amenity_bool(amenity_name) - val1.get_amenity_bool(amenity_name))
        how_good_a_swap_for_h2 = household2.get_amenity_importance(amenity_name)*(val1.get_amenity_bool(amenity_name) - val2.get_amenity_bool(amenity_name))

        amenities_value1 += how_good_a_swap_for_h1
        amenities_value2 += how_good_a_swap_for_h2

    amenity_names_personal = ['religion', 'language', 'age_category']
    for amenity_name in amenity_names_personal:

        category1 = household1.get_amenity_category(amenity_name)
        prop1isgood = get_proportion_of_neighbours(A, household1, loc1, category1, amenity_name)
        prop2isgood = get_proportion_of_neighbours(A, household2, loc2, category1, amenity_name)
        how_good_a_swap_for_h1 = household1.get_amenity_importance(amenity_name)*(prop2isgood - prop1isgood)  # positive if prop2 is better for h1

        category2 = household2.get_amenity_category(amenity_name)
        prop1isgood = get_proportion_of_neighbours(A, household1, loc1, category2, amenity_name)
        prop2isgood = get_proportion_of_neighbours(A, household2, loc2, category2, amenity_name)
        how_good_a_swap_for_h2 = household2.get_amenity_importance(amenity_name)*(prop1isgood - prop2isgood)  # positive if prop1 is better for h2

        amenities_value1 += how_good_a_swap_for_h1
        amenities_value2 += how_good_a_swap_for_h2

    overall_amenities_value = amenities_value1 + amenities_value2

    return overall_amenities_value

def move(n,m, A, V):
    
    # Propose a move

    # Flattened choice using numpy
    total_grid_cells = n * m

    # Randomly selects a number between 0 and the total number of grid cell
    flat_indices = np.random.choice(total_grid_cells, size=2, replace=False)  # replace=False to ensure distinct choices
    # Translate value back into distinct gridspace 
    # divmod does i1 = flat_indices // m, j1 = flat_indices % m
    i1, j1 = divmod(flat_indices[0], m)
    i2, j2 = divmod(flat_indices[1], m)
    # Ensure not landing on amenity location
    while the_map[i1][j1] is not None or the_map[i2][j2] is not None:
        # Randomly selects a number between 0 and the total number of grid cells
        flat_indices = np.random.choice(total_grid_cells, size=2, replace=False)  # replace=False to ensure distinct choices
        i1, j1 = divmod(flat_indices[0], m)
        i2, j2 = divmod(flat_indices[1], m)
    
    print("Random position 1:", (i1,j1))
    print("Random position 2:", (i2,j2))

    # Calculate value of Delta function given by the worksheet
    # delta for income vs house price
    A_incomes = get_incomes_matrix(A)
    V_prices = get_prices_matrix(V)
    # DEBUG: check for NaNs
    if np.isnan(A_incomes).any():
        print("NaN found in A_incomes")
    if np.isnan(V_prices).any():
        print("NaN found in V_prices")

    print("Values used in delta money:")
    print("A1, V1:", A_incomes[i1,j1], V_prices[i1,j1])
    print("A2, V2:", A_incomes[i2,j2], V_prices[i2,j2])

    d_money = (A_incomes[i1,j1] - V_prices[i1,j1])**2 + (A_incomes[i2,j2] - V_prices[i2,j2])**2 - (A_incomes[i1,j1] - V_prices[i2,j2])**2 - (A_incomes[i2,j2] - V_prices[i1,j1])**2 
    # Modify value of house to include amenities and optional amenities
    d_amenities = get_amenities_value(A, [i1, j1], V[i1,j1], A[i1,j1], [i2, j2], V[i2,j2], A[i2,j2])

    amenities_threshold = 0.02  # minimum threshold of delta amenities to allow swap
    
    # Swap inhabitants if delta value is positive
    if d_money > 0 and d_amenities > amenities_threshold:
        A[i1,j1], A[i2,j2] = A[i2,j2], A[i1,j1]
    
    print("Delta Value (money): ", d_money)
    print("Delta Value (amenities): ", d_amenities)
    #print('Updated income',A)
    #print('Updated values',V)
    return A, V


def createlat_(n,m, income_values, income_probabilities):
    # Create matrices to keep track of house prices and corresponding house owners

    # Define lattice size

    # Create lattice of house prices
    house_values = [100, 500, 1000] # Adjust accordingly
    house_probabilities = [0.5, 0.3, 0.2] # Adjust accordingly
    V_prices = np.random.choice(house_values, size = (n,m), p = house_probabilities)

    V = gen_amenity_matrix(V_prices, the_map)

    # Create lattice of house owners
    # Poor = 0.1
    # Middle = 0.5
    # Rich = 1
    A_incomes = np.random.choice(income_values, size=(n, m), p=income_probabilities)
    # create n x m array of Household objects
    A = np.zeros((n, m), dtype=object)
    for i in range(n):
        for j in range(m):
            A[i, j] = Household(float(A_incomes[i, j]))
            cell_type = the_map[i][j]

            if cell_type in amenities_names_fixed:
                # Amenity cells: no resident, show "amenity" colour in plots
                A[i, j].set_income(0.0001)

            elif cell_type is not None:
                # Parks, roads, station, hotel: completely blank / non-buildable
                A[i, j].set_income(0.0)

            # else: cell_type is None -> normal residential cell
            # keeps its drawn income (0.1, 0.5, or 1.0)


    #print("House Prices (V):\n", V)
    #print("\nIncome Levels (A):\n", A)
    return A, V

def one_step(A, V, neighbourhood_size, n, m):
    update_households, update_prices = house_update(neighbourhood_size,A,V)
    final_households, final_values = move(n,m,update_households, update_prices)
    return final_households, final_values

def p_steps(n,m, num_steps, households,value):
    House_history = []
    House_history.append(get_incomes_matrix(households).copy())
    for i in range(0,num_steps):
        households, value = one_step(households,value,house_range,n,m)
        House_history.append(get_incomes_matrix(households).copy())

    #print('Final incomes:', get_incomes_matrix(households), 'Final values:', value )
    
    return House_history

households,value_matrix = createlat_(n,m, income_values, income_probabilities)
#print("Values:", get_prices_matrix(value_matrix))
House_history = p_steps(n,m, num_steps, households,value_matrix)

# Define house for display
house = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
                    [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
                    [0, 0, 1, 1, 0, 1, 1, 0, 0, 0],
                    [0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
                    [1, 1, 0, 0, 0, 0, 0, 1, 1, 0],
                    [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
                    [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
                    [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
                    [0, 1, 1, 1, 1, 1, 1, 1, 0, 0]])

house_size = house.shape[0]
initial_lattice = House_history[0]
print("Initial Income Matrix:\n", initial_lattice)

def get_display_A(lattice, city_map):
    # note 'lattice' is 'House_history' element which is list of income matrices
    display_income = np.zeros((house.shape[0]*lattice.shape[0], house.shape[1]*lattice.shape[1]))
    for row in range(n):
        for col in range(m):
            cell_type = city_map[row, col]
            if cell_type == 'park':
                val = 0.0
            elif cell_type in amenities_names_fixed or cell_type == 'station':
                val = 0.0001
            else:

                val = lattice[row, col]             
            display_income[row*house_size:(row+1)*house_size, col*house_size:(col+1)*house_size] = lattice[row, col] * house

    return display_income

## Custom colour map with legend and no axis numbers
cmap = colors.ListedColormap(["#545454", "#6126b4", "#ffcc00", "#0099ff", "#ff0000"])  # change to whatever
bounds=np.concatenate(([0,0.00005, 0.00015], np.array(income_values)+0.01), axis=0)
norm = colors.BoundaryNorm(bounds, cmap.N)


initial_image = get_display_A(initial_lattice, the_map)
## Create plot
fig = plt.figure()
im=plt.imshow(initial_image, cmap=cmap, norm=norm)

legend_ticks = [(bounds[i+1]+bounds[i])/2 for i in range(0,len(bounds)-1)]
cbar = plt.colorbar(im, cmap=cmap, norm=norm, boundaries=bounds, ticks=legend_ticks)
cbar.ax.set_yticklabels(['Background', 'Amenity', 'Poor', 'Middle', 'Rich'])
plt.xticks([])
plt.yticks([])

def init():
    display_A = get_display_A(initial_lattice,the_map)
    im.set_data(display_A)
    return [im]

def animate(i):
    global House_history
    image = House_history[i]
    im.set_array(get_display_A(image, the_map))
    return [im]

print(len(House_history))

nSeconds = num_steps/8
fps = n*1.5
anim = animation.FuncAnimation(
                               fig, 
                               animate, 
                               init_func=init,
                               frames = len(House_history),
                               interval = 1000 / fps, # in ms
                               blit = True,
                               repeat=False
                               )

anim.save('housing_sim1.gif', fps=fps)

print("Finished and saved animation.")
plt.show()

In [ ]:
def plot_city_structure(city_map):
    type_to_int = {
        None: 0,
        'park': 1,
        'shop': 2,
        'supermarket': 3,
        'post office': 4,
        'gym': 5,
        'school': 6,
        'station': 7,
        'hotel': 8,
        'castle': 9,
        'university': 10,
    }

    n, m = city_map.shape
    Z = np.zeros((n, m), dtype=int)
    for i in range(n):
        for j in range(m):
            Z[i, j] = type_to_int.get(city_map[i, j], 0)

    cmap = colors.ListedColormap([
        "#f0f0f0",  # 0 background
        "#a0a0a0",  # 1 park/roads
        "#ffcc00",  # 2 shop
        "#ffa500",  # 3 supermarket
        "#ff66cc",  # 4 post office
        "#66ccff",  # 5 gym
        "#33aa33",  # 6 school
        "#0000ff",  # 7 station
        "#800080",  # 8 hotel
        "#333333",  # 9 castle
        "#99ffcc",  # 10 university
    ])

    plt.figure(figsize=(6,6))
    plt.imshow(Z, cmap=cmap, origin="upper")
    plt.xticks([])
    plt.yticks([])
    plt.title("Structural map: parks / roads / amenities")
    plt.colorbar()
    plt.show()

the_map = create_edinburgh_map(n, m)
plot_city_structure(the_map)